In [ ]:
#Import des packages necessaires à l'analyse

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from wordcloud import WordCloud
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import numpy as np
!pip install wordcloud

# Exploration de donnes et Visualisations

In [ ]:
#Import du DataFrame et affichage des premières lignes

filepath = "../csv/scraping_commentaires_4_banques.csv"
df=pd.read_csv(filepath)
df.head()

In [ ]:
#Informations du DataFrame
df.info()

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Télécharger les ressources nécessaires pour NLTK
nltk.download('stopwords')

# Créer une copie des données pour la manipulation
df_copy = df.copy()

# Concaténer tous les avis en une seule chaîne de caractères
all_comments = ' '.join(df_copy['Avis'].dropna())

# Tokenisation des commentaires
tokens = word_tokenize(all_comments)

# Suppression des stopwords
stop_words = set(stopwords.words('french')) # stopwords en français
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

# Création de la Wordcloud avec des options personnalisées
wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis',
                      contour_color='steelblue', contour_width=2,
                      font_path=None).generate(' '.join(filtered_tokens))

# Affichage de la Wordcloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Nuage de mots des commentaires')
plt.axis('off')
plt.show()

In [ ]:
# Le nettoyage et le traitement des données:
#Supprimer les doublons
df.drop_duplicates(inplace=True)

# Supprimer les valeurs manquantes
df.dropna(inplace=True)
df.info()

In [ ]:
# Supprimer le préfixe "Date de l'expérience: " de la colonne "Date de l'expérience"
df['Date de l\'expérience'] = df['Date de l\'expérience'].str.replace('Date de l\'expérience: ', '')

# Convertir la colonne "Date de publication" en objets datetime
df['Date de publication'] = pd.to_datetime(df['Date de publication'])

# Créer trois nouvelles colonnes pour le jour, le mois et l'année de publication
df['DayPub'] = df['Date de publication'].dt.day
df['MonthPub'] = df['Date de publication'].dt.month
df['YearPub'] = df['Date de publication'].dt.year

df.head(3)

In [ ]:
# Tokenisation et Lemmatization du DataFrame puis réaffichage du nuage de mots

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

def clean_text(text):
    
    # Suppression des caractères spéciaux et de la ponctuation
    processed_comments = re.sub(r"[^\w\s]", "", text)
    
    # Conversion en minuscules
    processed_comments = processed_comments.lower()
    
    # Tokenisation du texte
    tokens = word_tokenize(processed_comments)
    
    # Suppression des mots vides
    french_stopwords = set(stopwords.words("french"))
    custom_stopwords = ["boursorama", "monabanq", "hello", "bank", "fortuneo", "ça", "cette", "ca", "c'est", "en", 
                    "aussi", "ni", "tout", "déjà", "pour", "de", "et", "à", "le", "je", "la", "un", "pas", "que",
                    "les", "des", "une", "très", "si", "leurs", "aprè", "toute", "quand", "non", "peu", "jai", 
                    "cela", "ca", "cest", "a", "chez", "banque", "car", "donc"]
    french_stopwords.update(custom_stopwords)
    tokens = [word for word in tokens if word not in french_stopwords]
    
    # Lemmatisation des tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Joindre les tokens en une seule chaine
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

In [ ]:
# Trier les catégories de la variable cible
sorted_categories = df['Note'].value_counts().sort_index()

# Tracer le graphique avec les catégories triées
plt.figure(figsize=(8, 6))
sns.countplot(x='Note', data=df, order=sorted_categories.index)
plt.title('Distribution de la variable cible (Note)')
plt.xlabel('Note')
plt.ylabel("Nombre d'échantillons")
plt.show()

In [ ]:
# Importer la bibliothèque locale
import locale
# Définir la localisation en français
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')

# Extraire le jour de la semaine
df['DayOfWeek'] = df['Date de publication'].dt.strftime('%A')

df.head(3)

In [ ]:
# Répartition des commentaires par jour de la semaine
plt.figure(figsize=(8, 6))
sns.countplot(x='DayOfWeek', data=df, order=['lundi', 'mardi', 'mercredi', 'jeudi', 'vendredi', 'samedi', 'dimanche'])
plt.title('Répartition des commentaires par jour de la semaine')
plt.xlabel('Jour de la semaine')
plt.ylabel('Nombre de commentaires')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Répartition des commentaires par mois de l'année
mois_fr = ['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre']

plt.figure(figsize=(8, 6))
ax = df['MonthPub'].value_counts().sort_index().plot(kind='bar')
ax.set_title('Répartition des commentaires par mois de l\'année')
ax.set_xlabel('Mois')
ax.set_ylabel('Nombre de commentaires')
ax.set_xticklabels(mois_fr, rotation=45)
plt.show()

In [ ]:
#Répartition des commentaires par l'année et par banque à partir de 2017
# Filtrer les données à partir de 2017
data_2017 = df[df['YearPub'] >= 2017]

# Grouper les données par année et par banque
grouped_data = data_2017.groupby(['YearPub', 'Banque']).size().unstack(fill_value=0)

# Afficher le graphique à barres empilées
grouped_data.plot(kind='bar', stacked=True, figsize=(12, 8))
plt.title('Répartition des commentaires par année et par banque à partir de 2017')
plt.xlabel('Année')
plt.ylabel('Nombre de commentaires')
plt.xticks(rotation=45)
plt.legend(title='Banque', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# Feature engineering et Pre-processing 

In [ ]:
import string

# Longueur de la réponse
df['Longueur_commentaire'] = df['Avis'].apply(lambda x: len(x))

# Nombre de ponctuations
def count_punctuation(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['Nombre_ponctuations'] = df['Avis'].apply(lambda x: count_punctuation(x))

# Focalisation sur le nombre de points d'exclamation
def count_exclamation(text):
    count = text.count('!')
    return count

df['Nombre_points_exclamation'] = df['Avis'].apply(lambda x: x.count('!'))

df.head(3)

In [ ]:
import statsmodels.api as sm

# Effectuer le test ANOVA
model = sm.OLS.from_formula('Longueur_commentaire ~ Note', data=df).fit()
# Afficher les résultats de régression
print(model.summary())

In [ ]:
# Trier les données par la colonne 'Note'
df_sorted = df.sort_values(by='Note')

# Tracer le boxen plot avec les données triées
plt.figure(figsize=(10, 6))
sns.boxenplot(x='Note', y='Longueur_commentaire', data=df_sorted)
plt.title('Distribution de la longueur des commentaires en fonction de la note attribuée')
plt.xlabel('Note attribuée')
plt.ylabel('Longueur des commentaires')
plt.show()

In [ ]:
# Convertir la colonne "Note" en type numérique
df['Note'] = pd.to_numeric(df['Note'])
df.info()

In [ ]:
# Visualisation
sns.boxplot(x='Note', y='Nombre_ponctuations', data=df)
plt.title('Relation entre le nombre de ponctuations et la note du commentaire')
plt.show()

sns.boxplot(x='Note', y='Nombre_points_exclamation', data=df)
plt.title('Relation entre le nombre de points d\'exclamation et la note du commentaire')
plt.show()

In [ ]:
# Effectuer le test ANOVA pour Nombre_ponctuations
formula_ponctuations = 'Nombre_ponctuations ~ Note'
model_ponctuations = sm.OLS.from_formula(formula_ponctuations, data=df).fit()
print("Pour Nombre_ponctuations :")
print(model_ponctuations.summary())

# Effectuer le test ANOVA pour Nombre_points_exclamation
formula_exclamation = 'Nombre_points_exclamation ~ Note'
model_exclamation = sm.OLS.from_formula(formula_exclamation, data=df).fit()
print("\nPour Nombre_points_exclamation :")
print(model_exclamation.summary())

In [ ]:
pip install textblob-fr

In [ ]:
from textblob import TextBlob
from textblob_fr import PatternAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Appliquer le nettoyage du texte sur la colonne 'Avis'
df['Avis_nettoyé'] = df['Avis'].apply(clean_text)

# Créer la fonction pour calculer la polarité et la subjectivité en utilisant TextBlob avec le modèle de langue français
def calculer_sentiment_polarite_subjectivite_francais(post):
    blob = TextBlob(post, analyzer=PatternAnalyzer())
    polarite = blob.sentiment[0]
    subjectivite = blob.sentiment[1]
    
    # Déterminer le sentiment basé sur la polarité
    if polarite > 0:
        sentiment = "Positif"
    elif polarite < 0:
        sentiment = "Négatif"
    else:
        sentiment = "Neutre"
    
    return sentiment, polarite, subjectivite

# Appliquer la fonction calculer_sentiment_polarite_subjectivite_francais à la colonne 'Avis_nettoyé' de votre DataFrame
df['Sentiment'], df['Polarite'], df['Subjectivite'] = zip(*df['Avis_nettoyé'].apply(calculer_sentiment_polarite_subjectivite_francais))
df.head(5)

In [ ]:
df[['Polarite', 'Subjectivite']].describe()

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
sentiment_counts = df['Sentiment'].value_counts()

labels = sentiment_counts.index.tolist()
values = sentiment_counts.values.tolist()

plt.pie(values, labels=labels, autopct='%1.1f%%')
plt.title("Répartition des sentiments")
plt.show()

In [ ]:
import pandas as pd

# Convertir la colonne 'Date de publication' en objets datetime
df['Date de publication'] = pd.to_datetime(df['Date de publication'])

# Extraire uniquement la date
df['Date'] = df['Date de publication'].dt.date

# Afficher les premières lignes du DataFrame avec la nouvelle colonne 'Date'

# Tri du DataFrame par ordre croissant des index
df = df.sort_index(ascending=True)

# Affichage du DataFrame trié
df.head()

In [ ]:
#Appliquer le nettoyage du texte sur la colonne 'Avis'
df['Avis_nettoyé'] = df['Avis'].apply(clean_text)

# Concaténer tous les avis nettoyés en une seule chaîne de caractères
cleaned_comments = ' '.join(df['Avis_nettoyé'])

# Tokenisation des commentaires nettoyés
cleaned_tokens = word_tokenize(cleaned_comments)

# Création de la Wordcloud avec des options personnalisées
wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis',
                      contour_color='steelblue', contour_width=2,
                      font_path=None).generate(' '.join(cleaned_tokens))

# Affichage de la Wordcloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Nuage de mots des commentaires après nettoyage et lemmatisation')
plt.axis('off')
plt.show()

In [ ]:
from collections import Counter
import seaborn as sns

# Compter les occurrences de chaque mot dans les commentaires nettoyés
word_counter = Counter(' '.join(df['Avis_nettoyé']).split())

# Sélectionner les 15 mots les plus fréquents
top_15_words = word_counter.most_common(15)

# Extraire les mots et leurs fréquences
mots = [mot[0] for mot in top_15_words]
freq = [mot[1] for mot in top_15_words]

# Créer un histogramme avec seaborn
plt.figure(figsize=(10, 6))
sns.barplot(x=mots, y=freq)
plt.title('15 mots les plus fréquemment employés par les clients dans les commentaires')
plt.xlabel('Mots')
plt.ylabel('Fréquence')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Définir une fonction pour générer le nuage de mots
def generate_wordcloud(text, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(text))
    plt.figure(figsize=(10, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Grouper les avis par polarité
avis_par_polarite = {
    'Positif': df[df['Polarite'] > 0]['Avis_nettoyé'],
    'Négatif': df[df['Polarite'] < 0]['Avis_nettoyé'],
    'Neutre': df[df['Polarite'] == 0]['Avis_nettoyé']
}

# Générer les nuages de mots pour chaque type d'avis
for polarite, avis in avis_par_polarite.items():
    generate_wordcloud(avis, f'Nuage de mots - Avis {polarite.lower()}')

In [ ]:
import plotly.express as px

# Compter les occurrences de chaque mot dans les commentaires nettoyés
word_counter = Counter(' '.join(df['Avis_nettoyé']).split())

# Division des avis en tokens en fonction des catégories
tokens_positifs = [token for avis in df[df['Sentiment'] == 'Positif']['Avis_nettoyé'] for token in avis.split()]
tokens_negatifs = [token for avis in df[df['Sentiment'] == 'Négatif']['Avis_nettoyé'] for token in avis.split()]
tokens_neutres = [token for avis in df[df['Sentiment'] == 'Neutre']['Avis_nettoyé'] for token in avis.split()]

from collections import Counter
#Obtenir les mots les plus utilisés
def get_maxtoken(avis, num=30):
    word_tokens = Counter(avis)
    max_common = word_tokens.most_common(num)
    return dict(max_common)

def token_df_vis(x,title):
    df_token = pd.DataFrame(get_maxtoken(x).items(), columns=['words','count'])
    fig = px.bar(df_token, x='words', y='count', title = title)
    fig.show()

# Créer les histogrammes pour chaque catégorie d'avis
token_df_vis(tokens_positifs,'Positive')
token_df_vis(tokens_negatifs,'Negative')
token_df_vis(tokens_neutres,'Neutral')

# CountVectorizer : 

In [ ]:
pip install --upgrade scikit-learn

In [ ]:
pip install --upgrade --user scikit-learn

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Sélectionner les colonnes pertinentes
df_note = df[['Note', 'Avis_nettoyé']]

# Appliquer CountVectorizer sur les avis nettoyés
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(df_note['Avis_nettoyé'])

# Exploration de la matrice de comptage
print("Dimensions de la matrice de caractéristiques:", features.shape)
print("Vocabulaire:", list(vectorizer.vocabulary_.keys())[:10])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Diviser les données en ensembles d'entraînement et de test (80% pour l'entraînement, 20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(features, df['Note'], test_size=0.2, random_state=42)

# Afficher les dimensions des ensembles d'entraînement et de test
print("Dimensions de l'ensemble d'entraînement :", X_train.shape, y_train.shape)
print("Dimensions de l'ensemble de test :", X_test.shape, y_test.shape)

In [ ]:
# Définition du modèle RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1) 
# Entraînement du modèle
rf.fit(X_train, y_train)

# Affichage de la précision sur l'ensemble de test
accuracy = rf.score(X_test, y_test)
print("Précision sur l'ensemble de test :", accuracy)

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Appliquer TF-IDFVectorizer sur les avis nettoyés
tfidf_vectorizer = TfidfVectorizer()
features_tfidf = tfidf_vectorizer.fit_transform(df_note['Avis_nettoyé'])

# Exploration de la matrice TF-IDF
print("Dimensions de la matrice de caractéristiques avec TF-IDFVectorizer:", features_tfidf.shape)
print("Vocabulaire:", list(tfidf_vectorizer.vocabulary_.keys())[:10])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, df['Note'], test_size=0.2, random_state=42)

In [ ]:
# Définition du modèle RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1) 

# Entraînement du modèle
rf.fit(X_train, y_train)

# Affichage de la précision sur l'ensemble de test
accuracy = rf.score(X_test, y_test)
print("Précision sur l'ensemble de test avec TF-IDFVectorizer :", accuracy)

In [ ]:
count_words = return_ngram(df['Avis_nettoyé'], ngram_range=(3, 3))

plt.figure(figsize=(15, 10))
sns.barplot(x='count', y='word', data=count_words.head(10))
plt.show()

In [ ]:
count_words = return_ngram(df['Avis_nettoyé'], ngram_range=(6, 6))

plt.figure(figsize=(15, 10))
sns.barplot(x='count', y='word', data=count_words.head(10))
plt.show()

In [ ]:
import matplotlib.pyplot as plt


fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 20))

for note in range(1, 6):
    avis_note = df[df['Note'] == note]['Avis_nettoyé']
    vectorizer = CountVectorizer()
    features = vectorizer.fit_transform(avis_note)
    word_counts = features.sum(axis=0)
    word_names = vectorizer.get_feature_names_out()
    word_count_dict = dict(zip(word_names, word_counts.A1))
    
    sorted_words = sorted(word_count_dict.items(), key=lambda x: x[1], reverse=True)
    top_words = [word for word, _ in sorted_words[:10]]
    
    row = (note - 1) // 2
    col = (note - 1) % 2
    
    ax = axes[row, col]
    ax.barh(top_words, [word_count_dict[word] for word in top_words])
    ax.set_title(f"Top 10 mots - Note {note}")
    ax.set_xlabel("Nombre d'occurrences")
    ax.set_ylabel("Mots")

plt.tight_layout()
plt.show()

# Annex

In [ ]:
##ANALYSE PAR BANQUE
banque_counts = df['Banque'].value_counts()
banque_counts

In [ ]:
# Répartition des commentaires par banque
plt.figure(figsize=(10, 6))
sns.countplot(x='Banque', data=df)
plt.title('Répartition des commentaires par banque')
plt.xlabel('Banque')
plt.ylabel('Nombre de commentaires')
plt.xticks(rotation=45)
plt.show()

# Comparaison des notes attribuées aux différentes banques
# Calculer la moyenne des notes attribuées à chaque banque
mean_notes = df.groupby('Banque')['Note'].mean().sort_values()

# Créer un graphique à barres pour visualiser les moyennes des notes attribuées aux différentes banques
plt.figure(figsize=(10, 6))
mean_notes.plot(kind='bar', color='skyblue')
plt.title('Moyenne des notes attribuées aux différentes banques')
plt.xlabel('Banque')
plt.ylabel('Note moyenne')
plt.xticks(rotation=45)
plt.show()